### Pantera-capital-portfolio monthly momemtum strategy

In [2]:
# get category symbols
category = "pantera-capital-portfolio"
import requests
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "category": category,
    "order": "market_cap_desc",
    "per_page": 300,
    "page": 1
}

response = requests.get(url, params=params)
data = response.json()

# Extract the symbols
defi_symbols = [coin['symbol'].upper() + '/USDT' for coin in data]
print("DeFi Symbols:", defi_symbols)

DeFi Symbols: ['XRP/USDT', 'TON/USDT', 'DOT/USDT', 'NEAR/USDT', 'FIL/USDT', 'ARB/USDT', 'INJ/USDT', 'ATOM/USDT', 'ONDO/USDT', 'HNT/USDT', 'STRK/USDT', 'MINA/USDT', 'LUNC/USDT', 'ZEC/USDT', 'ROSE/USDT', 'LPT/USDT', '1INCH/USDT', 'AMP/USDT', 'LUNA/USDT', 'KSM/USDT', 'AEVO/USDT', 'ZRX/USDT', 'ANKR/USDT', 'BAT/USDT', 'API3/USDT', 'AUDIO/USDT', 'ICX/USDT', 'SUSHI/USDT', 'INST/USDT', 'WAXP/USDT', 'AMPL/USDT', 'BAL/USDT', 'ZENT/USDT', 'CVC/USDT', 'LQTY/USDT', 'POND/USDT', 'AUCTION/USDT', 'OMNI/USDT', 'DODO/USDT', 'KNC/USDT', 'CELR/USDT', 'MAV/USDT', 'AURORA/USDT', 'ACA/USDT', 'OGN/USDT', 'GUSD/USDT', 'STRD/USDT', 'FLIP/USDT', 'XCN/USDT', 'GEOD/USDT', 'DESO/USDT', 'OMG/USDT', 'TT/USDT', 'FOX/USDT', 'FUN/USDT', 'SD/USDT', 'ORDER/USDT', 'KIN/USDT', 'GFT/USDT', 'A8/USDT', 'PTU/USDT', 'MTRG/USDT', 'STRDY/USDT', 'PERC/USDT', 'KAR/USDT', 'HNS/USDT', 'REP/USDT', 'GENE/USDT', 'RPG/USDT', 'ANC/USDT', 'CPH/USDT', 'MIR/USDT', 'FLX/USDT', 'NOTE/USDT', 'LITH/USDT', 'ECO/USDT', 'ENG/USDT', 'VEGA/USDT', 'UNB

In [3]:
# Fetch aviable symbols from Bybit
import ccxt

exchange = ccxt.bybit()
exchange.load_markets()
all_symbols = exchange.symbols

# Filter to include only DeFi symbols available on the exchange
available_defi_symbols = [symbol for symbol in defi_symbols if symbol in all_symbols]
print("Available DeFi Symbols:", available_defi_symbols)

Available DeFi Symbols: ['XRP/USDT', 'TON/USDT', 'DOT/USDT', 'NEAR/USDT', 'FIL/USDT', 'ARB/USDT', 'INJ/USDT', 'ATOM/USDT', 'ONDO/USDT', 'HNT/USDT', 'STRK/USDT', 'MINA/USDT', 'LUNC/USDT', 'ROSE/USDT', '1INCH/USDT', 'LUNA/USDT', 'KSM/USDT', 'AEVO/USDT', 'ZRX/USDT', 'ANKR/USDT', 'BAT/USDT', 'ICX/USDT', 'SUSHI/USDT', 'WAXP/USDT', 'ZENT/USDT', 'OMNI/USDT', 'AURORA/USDT', 'ACA/USDT', 'FLIP/USDT', 'OMG/USDT', 'SD/USDT', 'ORDER/USDT', 'A8/USDT', 'PTU/USDT', 'GENE/USDT', 'FTT/USDT']


In [4]:
# Fetch ohlcv from Bybit
import numpy as np
import pandas as pd
import datetime as dt
import pytz
import plotly.express as px
import plotly.graph_objects as go

symbols = available_defi_symbols
timeframe = '1d'
limit = 252

# Create an empty list to store DataFrames for each symbol
data_frames = []

for symbol in symbols:
    try:
        ohlcv_data = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['symbol'] = symbol

        # Convert timestamp to datetime format for readability
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

        data_frames.append(df)
    except Exception as e:
        print(f"Could not fetch data for {symbol}: {e}")

# Concatenate all DataFrames into a single DataFrame
all_data = pd.concat(data_frames, ignore_index=True)
all_data.head()

,timestamp,open,high,low,close,volume,symbol
0,2024-03-10,0.6199,0.6277,0.5966,0.6081,1.214860e+08,XRP/USDT
1,2024-03-11,0.6081,0.7441,0.5812,0.7228,8.034530e+08,XRP/USDT
2,2024-03-12,0.7228,0.7329,0.6578,0.6880,4.895242e+08,XRP/USDT
3,2024-03-13,0.6880,0.7025,0.6684,0.6891,2.394229e+08,XRP/USDT
4,2024-03-14,0.6891,0.7075,0.6379,0.6691,4.721125e+08,XRP/USDT


In [5]:
# Check min max date
print(all_data['timestamp'].min(), all_data['timestamp'].max())

2024-03-10 00:00:00 2024-11-16 00:00:00


In [6]:
# Filter dublicates and unnecesary timestamps
# Ensure that 'timestamp' is in datetime format
all_data['timestamp'] = pd.to_datetime(all_data['timestamp'])

# Find the latest date in the data
latest_date = all_data['timestamp'].max()

# Calculate the cutoff date for 252 days before the latest date
cutoff_date = latest_date - pd.Timedelta(days=252)

# Filter the data to include only rows within the last 252 days
filtered_data = all_data[all_data['timestamp'] >= cutoff_date]

# Check the new date range to confirm
print("Filtered data range:", filtered_data['timestamp'].min(), "-", filtered_data['timestamp'].max())

# Identify duplicate entries based on 'timestamp' and 'symbol'
duplicates = filtered_data[filtered_data.duplicated(subset=['timestamp', 'symbol'], keep=False)]
print("Duplicate entries:")
duplicates.info()

# Remove duplicate entries, keeping only the first occurrence
filtered_data = filtered_data.drop_duplicates(subset=['timestamp', 'symbol'], keep='first')


Filtered data range: 2024-03-10 00:00:00 - 2024-11-16 00:00:00
Duplicate entries:
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  0 non-null      datetime64[ns]
 1   open       0 non-null      float64       
 2   high       0 non-null      float64       
 3   low        0 non-null      float64       
 4   close      0 non-null      float64       
 5   volume     0 non-null      float64       
 6   symbol     0 non-null      object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 0.0+ bytes


In [7]:
# get closed price data
df = filtered_data[['timestamp', 'close', 'symbol']]
df.head()

,timestamp,close,symbol
0,2024-03-10,0.6081,XRP/USDT
1,2024-03-11,0.7228,XRP/USDT
2,2024-03-12,0.6880,XRP/USDT
3,2024-03-13,0.6891,XRP/USDT
4,2024-03-14,0.6691,XRP/USDT


In [8]:
# pivot by timestamp and symbol
df_pivoted = df.pivot(index='timestamp', columns='symbol', values='close')

print(df_pivoted.head())
print(df_pivoted.tail())

symbol      1INCH/USDT  A8/USDT  ACA/USDT  AEVO/USDT  ANKR/USDT  ARB/USDT  \
timestamp                                                                   
2024-03-10      0.6425      NaN    0.1321        NaN    0.05013     2.068   
2024-03-11      0.6616      NaN    0.1449        NaN    0.05231     2.083   
2024-03-12      0.6582      NaN    0.1883        NaN    0.05824     2.017   
2024-03-13      0.6837      NaN    0.1991      3.324    0.05836     2.116   
2024-03-14      0.6567      NaN    0.1822      3.037    0.05654     2.016   

symbol      ATOM/USDT  AURORA/USDT  BAT/USDT  DOT/USDT  ...  PTU/USDT  \
timestamp                                               ...             
2024-03-10    13.1767          NaN    0.3562    10.266  ...    0.4463   
2024-03-11    13.7027          NaN    0.3720    11.039  ...    0.4466   
2024-03-12    13.4098          NaN    0.3717    10.736  ...    0.4191   
2024-03-13    14.1133          NaN    0.3754    11.372  ...    0.4185   
2024-03-14    13.6783 

In [9]:
# Calculate monthly returns
returns_df = df_pivoted.pct_change().resample("ME").agg(lambda x: (x+1).prod()-1)
# Convert the index of past_cum_return_df to timezone-aware UTC
returns_df.index = returns_df.index.tz_localize('UTC')
returns_df.tail(7)

symbol,1INCH/USDT,A8/USDT,ACA/USDT,AEVO/USDT,ANKR/USDT,ARB/USDT,ATOM/USDT,AURORA/USDT,BAT/USDT,DOT/USDT,...,PTU/USDT,ROSE/USDT,SD/USDT,STRK/USDT,SUSHI/USDT,TON/USDT,WAXP/USDT,XRP/USDT,ZENT/USDT,ZRX/USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-05-31 00:00:00+00:00,0.225856,0.000000,-0.035581,-0.358183,-0.085811,0.093842,-0.017103,0.000000,0.023520,0.086328,...,-0.150925,0.134799,0.109397,-0.039967,0.219307,0.227224,0.030813,0.033546,0.083517,0.123163
2024-06-30 00:00:00+00:00,-0.173077,0.000000,-0.319417,-0.416333,-0.276487,-0.282395,-0.188762,-0.324313,-0.190398,-0.110395,...,-0.201089,-0.025734,-0.183960,-0.413764,-0.278830,0.209809,-0.379199,-0.080178,-0.397609,-0.291458
2024-07-31 00:00:00+00:00,-0.148537,-0.421999,-0.074180,-0.145254,-0.074823,-0.181818,-0.137442,-0.054837,-0.020274,-0.132786,...,0.044304,-0.185096,-0.189620,-0.327536,-0.202864,-0.118807,-0.016133,0.308339,-0.112331,-0.005973
2024-08-31 00:00:00+00:00,-0.297504,-0.336469,-0.107858,-0.277208,-0.174592,-0.217656,-0.214241,-0.248370,-0.156234,-0.209540,...,-0.069464,-0.311930,0.015379,-0.232759,-0.164671,-0.197205,-0.249243,-0.091026,-0.163178,-0.207594
2024-09-30 00:00:00+00:00,0.163043,0.008724,0.119171,0.210120,0.198066,0.177043,0.035854,0.181266,0.096873,0.041324,...,-0.016533,0.379008,-0.143405,0.233146,0.437276,0.053117,0.163306,0.079830,0.241615,0.153051
2024-10-31 00:00:00+00:00,-0.115025,-0.243243,-0.137346,-0.185365,-0.142506,-0.128926,-0.103417,-0.119677,-0.103410,-0.107103,...,-0.011717,-0.156254,-0.286929,-0.154897,-0.147132,-0.151138,-0.108608,-0.166503,-0.252747,-0.050523
2024-11-30 00:00:00+00:00,0.253046,0.124286,0.112701,0.148670,0.314368,0.235294,0.312032,0.148457,0.283042,0.366414,...,-0.008247,0.287716,1.214689,0.247439,0.154971,0.151434,0.301361,1.119113,0.319240,0.197103


In [10]:
# Create a line plot with Plotly
fig = px.line(returns_df, x=returns_df.index, y=returns_df.columns, title="Monthly returns")
# Adjust the layout for a wider plot
fig.update_layout(width=1200, height=600, xaxis_title="Timestamp", yaxis_title="Returns")
# Show the interactive plot
fig.show()

In [11]:
# rolling cumulative for 6 months return
past_cum_return_df = (returns_df + 1).rolling(6).apply(np.prod) - 1

past_cum_return_df.tail(7)

symbol,1INCH/USDT,A8/USDT,ACA/USDT,AEVO/USDT,ANKR/USDT,ARB/USDT,ATOM/USDT,AURORA/USDT,BAT/USDT,DOT/USDT,...,PTU/USDT,ROSE/USDT,SD/USDT,STRK/USDT,SUSHI/USDT,TON/USDT,WAXP/USDT,XRP/USDT,ZENT/USDT,ZRX/USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-31 00:00:00+00:00,-0.627704,-0.616478,-0.561696,-0.902497,-0.525633,-0.751451,-0.653715,-0.519983,-0.542111,-0.585135,...,-0.552767,-0.683521,-0.521819,-0.853317,-0.721973,0.923787,-0.681915,-0.068903,-0.515160,-0.637148
2024-09-30 00:00:00+00:00,-0.549547,-0.613133,-0.645708,-0.868057,-0.522701,-0.635981,-0.615435,-0.432973,-0.468193,-0.540224,...,-0.494333,-0.440358,-0.562347,-0.798531,-0.575661,0.053136,-0.654491,-0.028135,-0.398015,-0.560100
2024-10-31 00:00:00+00:00,-0.375919,-0.707235,-0.476592,-0.771851,-0.481096,-0.484848,-0.498093,-0.500833,-0.326333,-0.384041,...,-0.359313,-0.278703,-0.544987,-0.697390,-0.282267,-0.061077,-0.509848,0.017572,-0.550165,-0.313742
2024-11-30 00:00:00+00:00,-0.362076,-0.670849,-0.396117,-0.591678,-0.253950,-0.418231,-0.330024,-0.426728,-0.155519,-0.225229,...,-0.251653,-0.181507,-0.091658,-0.606797,-0.320138,-0.119063,-0.381202,1.086360,-0.452302,-0.268565


In [12]:
# Show cumulative return from past 6 months
fig = px.line(past_cum_return_df, x=past_cum_return_df.index, y=past_cum_return_df.columns, title="Rolling cumulative for 6 months return")
fig.update_layout(width=1200, height=600, xaxis_title="Timestamp", yaxis_title="Cumulative Returns")
fig.show()

In [13]:
# Define formation and measurement period
# Define the dates with timezone information
end_of_measurement_period = dt.datetime(2024, 9, 30, tzinfo=pytz.UTC)
formation_period = dt.datetime(2024, 10, 31, tzinfo=pytz.UTC)

end_of_measurement_period_return_df = past_cum_return_df.loc[end_of_measurement_period]
# Transpose the DataFrame so that tickers become rows and returns become a single column
end_of_measurement_period_return_df = end_of_measurement_period_return_df.T
end_of_measurement_period_return_df = end_of_measurement_period_return_df.reset_index()
# Rename columns for clarity
end_of_measurement_period_return_df.columns = ['Symbol', 'Return']
end_of_measurement_period_return_df.head()

,Symbol,Return
0,1INCH/USDT,-0.549547
1,A8/USDT,-0.613133
2,ACA/USDT,-0.645708
3,AEVO/USDT,-0.868057
4,ANKR/USDT,-0.522701


In [14]:
# highest momentum in the positive direction
end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmax()]

Symbol    HNT/USDT
Return    0.264008
Name: 14, dtype: object

In [15]:
# highest momentum in the negative direction
end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmin()]

Symbol    AEVO/USDT
Return    -0.868057
Name: 3, dtype: object

In [16]:
# rank symbols with quantiles
end_of_measurement_period_return_df['rank'] = pd.qcut(end_of_measurement_period_return_df.iloc[:,1], 11, labels=False)
end_of_measurement_period_return_df.head(7)

,Symbol,Return,rank
0,1INCH/USDT,-0.549547,5
1,A8/USDT,-0.613133,3
2,ACA/USDT,-0.645708,2
3,AEVO/USDT,-0.868057,0
4,ANKR/USDT,-0.522701,6
5,ARB/USDT,-0.635981,2
6,ATOM/USDT,-0.615435,3


In [17]:
# Create a bar plot to visualize ranks
fig = px.bar(
    end_of_measurement_period_return_df, 
    x='Symbol', 
    y='Return', 
    color='rank',  # Color by rank to visually distinguish groups
    title="Asset Returns and Ranks",
    labels={'rank': 'Rank'},
)

# Customize layout for readability
fig.update_layout(
    xaxis_title="Symbol",
    yaxis_title="Return",
    coloraxis_colorbar=dict(title="Rank"),
    width=1000,
    height=600
)

fig.show()


In [18]:
# select highest rank
long_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==10,"Symbol"].values
long_stocks

array(['FTT/USDT', 'HNT/USDT', 'TON/USDT', 'XRP/USDT'], dtype=object)

In [19]:
# select lowest rank
short_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==0,"Symbol"].values
short_stocks

array(['AEVO/USDT', 'FLIP/USDT', 'OMG/USDT', 'STRK/USDT'], dtype=object)

In [20]:
# long strategy return
from dateutil.relativedelta import relativedelta

long_return_df = returns_df.loc[formation_period + relativedelta(months=1), \
                                   returns_df.columns.isin(long_stocks)]
long_return_df

symbol
FTT/USDT    0.145378
HNT/USDT   -0.065342
TON/USDT    0.151434
XRP/USDT    1.119113
Name: 2024-11-30 00:00:00+00:00, dtype: float64

In [21]:
# short strategy return
short_return_df = returns_df.loc[formation_period + relativedelta(months=1), \
                                   returns_df.columns.isin(short_stocks)]
short_return_df

symbol
AEVO/USDT    0.148670
FLIP/USDT    0.150314
OMG/USDT     0.263404
STRK/USDT    0.247439
Name: 2024-11-30 00:00:00+00:00, dtype: float64

In [23]:
# Define Strategy tester class
class strategyTester:
    """Calculate metrics based on cumulative return"""

    def __init__(self, df: pd.Series, period: str, riskfree_rate: float = 0.03, initial_wealth: int = 1000):
        self.annualized_return_value = self.annualized_return(df, period)
        self.annualized_volatility_value = self.annualized_volatility(df, period)
        self.sharpe_ratio_value = self.sharpe_ratio(df, riskfree_rate)
        self.max_drawdown_value = self.max_drawdown(df, initial_wealth)
        
    # annualized return
    @staticmethod
    def annualized_return(df: pd.Series, period: str):
        if period == 'daily':
            factor = 252
        elif period == 'weekly':
            factor = 52
        elif period == 'monthly':
            factor = 12
        else:
            raise ValueError("Invalid period. Use 'daily', 'weekly', or 'monthly'.")
        
        annualized_return = (1 + df) ** (factor / len(df)) - 1
        print(f"Annualized return: {annualized_return}")
        return annualized_return

    # annualized volatility
    @staticmethod
    def annualized_volatility(df: pd.Series, period: str):
        if period == 'daily':
            factor = 252
        elif period == 'weekly':
            factor = 52
        elif period == 'monthly':
            factor = 12
        else:
            raise ValueError("Invalid period. Use 'daily', 'weekly', or 'monthly'.")
        
        volatility = df.std() * (factor ** 0.5)
        print(f"Annualized volatility: {volatility}")
        return volatility

    # Sharpe ratio
    @staticmethod
    def sharpe_ratio(df: pd.Series, riskfree_rate: float = 0.03):
        excess_return = df.mean() - riskfree_rate
        volatility = df.std()
        sharpe_ratio = excess_return / volatility if volatility != 0 else np.nan
        print(f"Sharpe ratio: {sharpe_ratio}")
        return sharpe_ratio

    # Drawdown calculation
    @staticmethod
    def drawdown(return_series: pd.Series, initial_wealth: float = 100):
        prior_peaks_series = return_series.cummax()
        drawdown_series = (return_series - prior_peaks_series) / prior_peaks_series
        return pd.DataFrame({
            "Wealth Index": return_series,
            "Prior Peaks": prior_peaks_series,
            "Drawdown": drawdown_series
        })
    
    # Max drawdown
    @staticmethod
    def max_drawdown(df: pd.Series, initial_wealth):
        drawdown_df = strategyTester.drawdown(df, initial_wealth)
        max_drawdown = drawdown_df["Drawdown"].min()
        print(f"Max drawdown: {max_drawdown} with initial wealth: {initial_wealth}")
        return max_drawdown


In [24]:
# long momentum strategy test
long_momentum_strategy_test = strategyTester(long_return_df, 'monthly', 0.03, 1000)

Annualized return: symbol
FTT/USDT    0.502613
HNT/USDT   -0.183496
TON/USDT    0.526571
XRP/USDT    8.516174
Name: 2024-11-30 00:00:00+00:00, dtype: float64
Annualized volatility: 1.8381858323853406
Sharpe ratio: 0.5797653710852276
Max drawdown: -1.4494611915905513 with initial wealth: 1000


In [25]:
# short momentum strategy test
short_momentum_strategy_test = strategyTester(short_return_df, 'monthly', 0.03, 1000)

Annualized return: symbol
AEVO/USDT    0.515604
FLIP/USDT    0.522120
OMG/USDT     1.016634
STRK/USDT    0.941147
Name: 2024-11-30 00:00:00+00:00, dtype: float64
Annualized volatility: 0.21307247108208466
Sharpe ratio: 2.803777388779988
Max drawdown: -0.060609887262734564 with initial wealth: 1000


In [22]:
# Comparison of monthly Long and Short momentum strategies
# Initialize an empty figure
fig = go.Figure()

# Add the first strategy (e.g., long strategy)
fig.add_trace(go.Bar(x=long_return_df.index, y=long_return_df.values, marker_color='blue', name="Long"))

# Add the second strategy (e.g., short strategy)
fig.add_trace(go.Bar(x=short_return_df.index, y=short_return_df.values, marker_color='red', name="Short"))

# Update layout for title and axis labels
fig.update_layout(
    title="Comparison of monthly Long and Short momentum strategies",
    width=1400,
    height=500,
    xaxis_title="Symbol",
    yaxis_title="Cumulative Returns",
    legend_title="Strategies",
)

# Show the plot
fig.show()

In [26]:
# Annualized Return and Max Drawdown from Long and Short Strategies
# Initialize an empty figure
fig = go.Figure()

# Add traces for Long strategy
fig.add_trace(go.Bar(
    x=["Annualized Return (Long)", "Max Drawdown (Long)"], 
    y=[long_momentum_strategy_test.annualized_return_value.mean(), long_momentum_strategy_test.max_drawdown_value], 
    marker_color='blue', 
    name="Long"
))

# Add traces for Short strategy
fig.add_trace(go.Bar(
    x=["Annualized Return (Short)", "Max Drawdown (Short)"], 
    y=[short_momentum_strategy_test.annualized_return_value.mean(), short_momentum_strategy_test.max_drawdown_value], 
    marker_color='red', 
    name="Short"
))

# Update layout
fig.update_layout(
    title="Annualized Return and Max Drawdown from Long and Short Strategies",
    xaxis_title="Metrics",
    yaxis_title="Values",
    barmode='group',  # Group bars by category
    width=800,
    height=400
)

# Show the plot
fig.show()



In [27]:
# Annualized Volatility and Max Drawdown from Long and Short Strategies
# Create a figure
fig = go.Figure()

# Add bars for the Long Strategy - Volatility and Drawdown
fig.add_trace(go.Bar(
    x=["Annualized Volatility(long)", "Sharpe Ratio(long)"], 
    y=[long_momentum_strategy_test.annualized_volatility_value, long_momentum_strategy_test.sharpe_ratio_value],
    name="Long",
    marker_color='blue'
))

# Add bars for the Short Strategy - Volatility and Drawdown
fig.add_trace(go.Bar(
    x=["Annualized Volatility(short)", "Sharpe Ratio(short)"], 
    y=[short_momentum_strategy_test.annualized_volatility_value, short_momentum_strategy_test.sharpe_ratio_value],
    name="Short",
    marker_color='red'
))

# Update layout for grouped bars
fig.update_layout(
    title="Annualized Volatility and Max Drawdown from Long and Short Strategies",
    xaxis_title="Metrics",
    yaxis_title="Metric Value",
    barmode='group',  # Group bars side-by-side
    width=800,
    height=400
)

# Show the plot
fig.show()
